In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from   dolfin import *
from   MyMods import FilePrep as fpr
from   MyMods import CustomBC as cbc
from   MyMods import CustomPropertyAssign as cpa

We begin by importing some useful standard modules such as Numpy and Matplotlib.
The entirety of FEniCS modules is imported by giving
```python 
from dolfin import *
```
Furthermore, we must import some custom modules that were created for this course.
- `FilePrep` handles file conversions and exportation
- `CustomBC` provides some methods for easy assignment of boundary conditions
- `CustomPropertyAssign` provides some methods to assign different properties to different subdomains

### The script:

In [2]:
#fname       = fpr.OpenGMSHGeom()
fname = 'corner_central_inclusion'
mesh        = Mesh(fname + ".xml")
subdomains  = MeshFunction("size_t", mesh, fname + "_physical_region.xml")      
boundaries  = MeshFunction("size_t", mesh, fname + "_facet_region.xml")
ds          = Measure('ds', domain=mesh, subdomain_data=boundaries)
dx          = Measure('dx', domain=mesh, subdomain_data=subdomains)
volume      = 1

RuntimeError: 

*** -------------------------------------------------------------------------
*** DOLFIN encountered an error. If you are not able to resolve this issue
*** using the information listed below, you can ask for help at
***
***     fenics-support@googlegroups.com
***
*** Remember to include the error message listed below and, if possible,
*** include a *minimal* running example to reproduce the error.
***
*** -------------------------------------------------------------------------
*** Error:   Unable to read data from XML file.
*** Reason:  Unable to open file "corner_central_inclusion.xml".
*** Where:   This error was encountered inside XMLFile.cpp.
*** Process: 0
*** 
*** DOLFIN version: 2019.1.0
*** Git changeset:  
*** -------------------------------------------------------------------------


To load a geometry, we call the function `OpenGMSHGeom()`. This function contains a call to a graphical file explorer that allows to select a '.geo' file. Once the selection is done, the function takes care of sending the command to mesh the geometry using Gmsh, and converting the data to subdomains and boundaries information. All mesh files are stored in a subfolder named 'mesh_files'.
The `Measure` class is called to create integration partitions. We create the differential `ds` for the boundaries and `dx` for the elements surface. We also state that the domain "volume" is unitary.

We can plot the mesh to see the subdomains:

In [ ]:
plt.figure(1)
plot(mesh, linewidth=0.5, color='black')
p = plot(subdomains, title='Subdomains')
plt.colorbar(p)

As can be observed from the colorbar, the center inclusion is assigned ID=1, the remaining subdomain has ID=0

In [ ]:
k_values    = [10, 50, 1]
k           = cpa.AssignSubPropertiesScalar(mesh, subdomains, k_values)

We assign the material parameter $k$ to the subdomains. The assignment is made by creating a Numpy array where the value $i$ will be assigned to the subdomain $i$.
Once the values have been entered, they are assigned to the mesh elements using the function `AssignSubPropertiesScalar(mesh, subdomains, k_values)`

The weak formulation of Poisson's equation elasticity reads:

\begin{equation}
\int_{\Omega} k \nabla u \cdot \nabla v \: dx = \int_{\Omega} f \cdot v \: dx
\end{equation}

Which is often expressed as:

\begin{equation}
a(u,v) = L(v)
\end{equation}

The solution we seek, $u$, is a scalar field, so we create a suitable function space. In this function space we use some Dolfin classes to create the trial function $u$ and the test function $v$.

In [ ]:
V           = FunctionSpace(mesh, 'P', 1)
u           = TrialFunction(V)
v           = TestFunction(V)

### Left hand side: $a(u,v)$

The left hand side of the weak form equation becomes simply:

In [ ]:
a = k*dot(grad(u), grad(v))*dx

### Right hand side: $L(v)$
We can define and a source term $f$ acting on the volume (in this example, $f=0$):

In [ ]:
f = Constant(0.0)
L = f*v*dx

### Boundary conditions
We create a list named `bcs`, where we will store all the Dirichlet boundary conditions. Each Dirchlet BC is created as an istance of the class `DirichletBC()`.

In [ ]:
bcs = [DirichletBC(V, 0.0, boundaries, 1),                                      
       DirichletBC(V, 1.0, boundaries, 3)]

<img src="BC_scheme.PNG" alt="Drawing" style="width: 200px;" align="right" />


The second argument expresses the value of $u$ assigned. The third and fourth arguments specify the subdomain where the condition is applied. In this case, $u=0$ is imposed on the left edge (`boundaries, 1`), and $u=1$ is imposed on the right edge (`boundaries, 3`).

### Solution
We are now ready to solve the problem. We simply call the FEnicS function `solve`:

In [ ]:
u = Function(V)
solve(a == L, u, bcs)

### Plotting:
Plotting the solution is accomplished by calling the `plot`function of FEniCS.

In [ ]:
plt.figure(2)
p = plot(u, title='Solution U')
plt.colorbar(p)